In [235]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import geopandas as gpd
import random
from scipy.stats import truncnorm, norm
from collections import defaultdict
import math
from shapely.ops import transform
from pyproj import Geod
import pyarrow
from datetime import datetime
import sys
import importlib
from dataclasses import dataclass
import yaml
from typing import Union

In [4]:
# добавляем папку modules в sys.path, чтобы импортировать функции созданные ранее

# sys.path.append(os.path.abspath(os.path.join("..", "modules")))

In [5]:
# Функции генерации паттернов времени и базовые функции времени созданные ранее
import data_generator.fraud.time
import data_generator.fraud.txndata
importlib.reload(data_generator.fraud.time)
importlib.reload(data_generator.fraud.txndata)
from data_generator.general_time import *
from data_generator.utils import build_transaction
from data_generator.fraud.txndata import FraudTxnPartData, TransAmount, sample_category
from data_generator.fraud.time import derive_from_last_time

In [10]:
np.set_printoptions(suppress=True)
pd.set_option('display.max_columns', None)

In [7]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01\\notebooks'

In [8]:
os.chdir("..")

In [9]:
os.getcwd()

'C:\\Users\\iaros\\My_documents\\Education\\projects\\fraud_detection_01'

## Загрузка конфигураций

In [18]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

# Доп. данные для генерации фрод транзакций

**Загрузка данных:**
- оффлайн мерчантов
- онлайн мерчантов
- девайсов клиентов и мошенников
- городов с полигонами
- городов с координатами центров
- клиентов
- фрод IP адресов
- категорий и их характеристик
- Счетов клиентов
- Внешних счетов

In [11]:
offline_merchants = gpd.read_file("./data/cleaned_data/offline_merchants_points.gpkg")
online_merchant_ids = pd.read_csv("./data/cleaned_data/online_merchant_ids.csv").iloc[:, 0]
client_devices = pd.read_csv("./data/cleaned_data/client_devices.csv")
fraud_devices = pd.read_csv("./data/cleaned_data/fraud_devices.csv")
districts_ru = gpd.read_file("./data/cleaned_data/district_ru.gpkg")
area_centers = gpd.read_file("./data/cleaned_data/area_centers.gpkg")
clients_with_geo = gpd.read_file("./data/cleaned_data/clients_with_geo.gpkg") 
fraud_ips = gpd.read_file("./data/cleaned_data/fraud_ips.gpkg")
cat_stats_full = pd.read_csv("./data/cleaned_data/cat_stats_full.csv")
cat_fraud_amts = pd.read_csv("./data/cleaned_data/cat_fraud_amts.csv")
accounts = pd.read_csv("./data/generated_data/accounts.csv")
outer_accounts = pd.read_csv("./data/generated_data/outer_accounts.csv")

**Датафрейм под транзакции. Этап тестов**<br>
Создадим пустой датафрейм под транзакции. Будем копировать и использовать его при тестах функций.

In [13]:
transactions = pd.DataFrame(
            {"client_id": pd.Series(dtype="int64"),
            "txn_time": pd.Series(dtype="datetime64[ns]"),
             "unix_time":pd.Series(dtype="int64"),
            "amount": pd.Series(dtype="float64"),
            "type": pd.Series(dtype="string"),
            "channel": pd.Series(dtype="string"),
            "category": pd.Series(dtype="string"),
            "online":pd.Series(dtype="bool"),
            "merchant_id":pd.Series(dtype="int64"),
             "trans_city":pd.Series(dtype="string"),
            "trans_lat":pd.Series(dtype="float64"),
             "trans_lon":pd.Series(dtype="float64"),
            "trans_ip":pd.Series(dtype="string"),
             "device_id":pd.Series(dtype="int64"),
             "account": pd.Series(dtype="int64"),
            "is_fraud": pd.Series(dtype="bool"),
             "is_suspicious": pd.Series(dtype="bool"),
            "status":pd.Series(dtype="string"),
            "rule":pd.Series(dtype="string")})
transactions

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule


**Функция получения случайных значений из `truncnorm` распределения**
- пойдет в модуль `utils`

In [14]:
def get_values_from_truncnorm(low_bound, high_bound, mean, std, size=1):
    """
    Сгенерировать массив чисел из обрезанного нормального распределения.
    Можно сгенерировать массив с одним числом
    ------------
    low_bound - float, int. Нижняя граница значений
    high_bound - float, int. Верхняя граница значений 
    mean - float, int. Среднее
    std - float, int. Стандартное отклонение
    size - Количество чисел в возвращаемом массиве
    ------------
    Возвращает np.ndarray
    """
    return truncnorm.rvs((low_bound - mean) / std, (high_bound - mean) / std, loc=mean, scale=std, size=size)

## Класс `DropDistributorCfg`
- конфигурации на основании которых будут генерироваться транзакции дропов
  занятых распределением полученных денег: переводами, снятиями, покупкой криптовалюты.

In [52]:
# ----------- ВСТАВИТЬ ГОТОВЫЙ КЛАСС СЮДА -------------------------

### Создание аргументов для `DropDistributorCfg`

In [16]:
# timestamp-ы использованные еще для генерации легальных и purchase фрод транзакций
timestamps = pd.read_parquet("./data/generated_data/timestamps.parquet", engine="pyarrow")

In [19]:
time_weight_args = time_cfg["time_weights_args"]

In [20]:
time_weights_dict = get_all_time_patterns(time_weight_args)

**Легальные транзакции**  
Нужно знать их количество для определения числа фрод транзакций  
и следовательно, количества клиентов для их генерации

In [21]:
legit_txns = pd.read_parquet("./data/generated_data/legit_trans_148K.parquet", engine="pyarrow")

**Клиенты для дроп фрода**  
Случайно возьмем клиентов, у которых нет никаких транзакций  
Для этого загрузим семплированных для легальных транзакций клиентов, т.е. у кого были созданы легальные транзакции.  
У части этих клиентов также созданы purchase фрод транзакции. Всех этих клиентов надо исключить.

In [22]:
clients_sample_df = gpd.read_file("./data/cleaned_data/clients_sample.gpkg")

In [23]:
clients_sample_df.shape

(3000, 12)

Конфиги загружаются временно тут

In [316]:
# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [24]:
# доля дроп фрода - Дропы занимающиеся переводами и снятиями; и дропы, которые покупают товары на присланные деньги
fraud_rate = fraud_cfg["fraud_rates"]["total"]
drop_dist_share = drops_cfg["distributor"]["rate"] # Дропы распределители. Доля от всего фрода
drop_purch_share = drops_cfg["purchaser"]["rate"] # Дропы покупатели. Доля от всего фрода

In [25]:
# лимит на исходящие транзакции дропа. Нужен сейчас для вычисления количества дропов распределителей

dist_out_lim = drops_cfg["distributor"]["out_lim"]

In [33]:
# отсюда посчитаем количество клиентов для дроп фрода с распределением денег

legit_count = legit_txns.shape[0]
# подсчет количества транзакций равных 1% от всех транзакций
# т.к. не все транзакции еще созданные, то считаем основываясь на количестве легальных транзакций и fraud rate
one_perc = round(legit_count / ((1 - fraud_rate) * 100))
# Один процент транзакций умножаем на долю транзакций дропов-распределителей и делим на максимальное количество
# исходящих транзакций которое распределитель может сделать до детекта. Так находим сколько примерно дропов будет под такой фрод
dist_drops_count = round(one_perc * drop_dist_share / dist_out_lim) 
dist_drops_count

28

In [34]:
# Клиенты незадействованные ранее

not_used_clients = clients_with_geo.loc[~clients_with_geo.client_id.isin(clients_sample_df.client_id)].copy()

In [35]:
# семплируем dist_drops_count клиентов

dist_drop_clients = not_used_clients.sample(n=dist_drops_count).reset_index(drop=True)

In [36]:
dist_drop_clients.head(2)

,client_id,district_id,birth_date,sex,region,area,timezone,lat,lon,population,home_ip,geometry
0,52,32,1924-12-12,male,Курская,Курск,UTC+3,51.730339,36.192645,414595,2.60.0.50,"MULTIPOLYGON (((36.0603 51.67692, 36.06039 51...."
1,403,42,1973-05-26,female,Астраханская,Астрахань,UTC+4,46.365581,48.055998,520662,2.60.1.127,"MULTIPOLYGON (((47.87112 46.26966, 47.87127 46..."


In [37]:
dist_drop_clients.shape

(28, 12)

## Объект датакласса `DropDistributorCfg` конфиги для дропов распределителей

In [320]:
# Импорт на время разработки
import data_generator.configs
importlib.reload(data_generator.configs)
from data_generator.configs import DropDistributorCfg

**Доп. конфиги из `drop.yaml`**

In [317]:
# временно

# Общие настройки
with open("./config/base.yaml") as f:
    base_cfg = yaml.safe_load(f)
# Настройки фрода
with open("./config/fraud.yaml") as f:
    fraud_cfg = yaml.safe_load(f)
# Настройки фрода для дропов
with open("./config/drops.yaml", encoding="utf8") as f:
    drops_cfg = yaml.safe_load(f)
# Настройки времени
with open("./config/time.yaml") as f:
    time_cfg = yaml.safe_load(f)

In [318]:
# Параметры обозначенные в config/drops.yaml

dist_cfg = drops_cfg["distributor"]
period_in_lim = dist_cfg["period_in_lim"]
period_out_lim = dist_cfg["period_out_lim"]
dist_lag_int = drops_cfg["lag_interval"]
two_way_delta = drops_cfg["two_way_delta"]
pos_delta = drops_cfg["pos_delta"]
split_rate = dist_cfg["split_rate"]
chunks = dist_cfg["chunks"]
dist_inbound = drops_cfg["inbound_amt"]
dist_round = dist_cfg["round"]
dist_trf_lim = dist_cfg["trf_lim"]
dist_att = dist_cfg["attempts"]

In [322]:
dist_configs = DropDistributorCfg(clients=dist_drop_clients, timestamps=timestamps, accounts=accounts, \
                                  outer_accounts=outer_accounts, client_devices=client_devices, atms=None, \
                                  online_merchant_ids=online_merchant_ids, cities=districts_ru, period_in_lim=period_in_lim, \
                                  period_out_lim=period_out_lim, lag_interval=dist_lag_int, two_way_delta=two_way_delta, \
                                  pos_delta=pos_delta, split_rate=split_rate, chunks=chunks, inbound_amt=dist_inbound, round=dist_round, \
                                  trf_lim=dist_trf_lim, attempts=dist_att
                                 )

## Класс `DropPurchaserCfg` конфиги для дропов покупателей

In [237]:
@dataclass
class DropPurchaserCfg: # <-------------------- in development. Совсем не откорректирован.
    """
    Это данные на основе которых будут генерироваться транзакции дропов-покупателей
    ---------------------
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    accounts: pd.DataFrame. Номера счетов клиентов. 
    outer_accounts: pd.Series. Номера внешних счетов для транзакций вне банка.
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int. Количество входящих транзакций после которых дроп уходит на паузу.
    period_out_lim: int. Количество исходящих транзакций после которых дроп уходит на паузу.
    lag_interval: int. Минуты. На сколько дроп должен делать паузу после
                       достижения лимита входящих и/или исходящих транзакций.
                       Например 1440 минут(сутки). Отсчет идет от первой транзакции в последнем периоде активности.
    two_way_delta: dict. Минимум и максимум дельты времени. Для случаев когда дельта может быть и положительной и отрицательной.
                         Эта дельта прибавляется к lag_interval, чтобы рандомизировать время возобновления активности,
                         чтобы оно не было ровным. Берется из конфига drops.yaml
    pos_delta: dict. Минимум и максимум случайной дельты времени в минутах. Для случаев когда дельта может быть только положительной.
                          Эта дельта - промежуток между транзакциями дропа в одном периоде. Просто прибавляется ко времени последней транзакции.
    chunks: dict. Характеристики для генератора сумм транзакций по частям.
    inbound_amt: dict. Настройки для сумм входящих транзакций
    round: int. Округление целой части сумм транзакций. Напр. 500 значит что суммы будут кратны 500 - кончаться на 500 или 000
    """
    clients: pd.DataFrame
    timestamps: pd.DataFrame
    transactions: pd.DataFrame
    accounts: pd.DataFrame
    outer_accounts: pd.Series
    client_devices: pd.DataFrame
    offline_merchants: pd.DataFrame
    categories: pd.DataFrame
    online_merchant_ids: pd.Series
    time_weights_dict: dict
    rules: pd.DataFrame
    cities: pd.DataFrame
    fraud_amounts: pd.DataFrame
    period_in_lim: int
    period_out_lim: int
    lag_interval: int
    two_way_delta: dict
    pos_delta: dict
    chunks: dict
    inbound_amt: dict
    round: dict

## Класс `DropTxnPartData` - генерация части данных транзакции
-  локация, мерчант id, ip адрес, device id, канал, тип транзакции. 

In [ ]:
class DropTxnPartData:
    """
    Класс для генерации части данных о транзакции дропа:
    канал, тип операции, мерчант, геопозиция, город, IP адрес, иногда статус.
    ------------------
    Атрибуты:
    --------
    atms - pd.DataFrame. id банкоматов и их координаты.
    client_info - pd.DataFrame или namedtuple. Запись с информацией о клиенте
    online_merchant_ids- pd.Series. id онлайн мерчантов
    client_devices - pd.DataFrame. Девайсы клиентов.
    last_txn - tuple. Для кэширования предыдущей транзакции
    """
    def __init__(self, configs: Union[DropDistributorCfg, DropPurchaserCfg]):
        """
        configs: Один из датаклассов — DropDistributorCfg или DropPurchaserCfg.
                 Содержит параметры и конфиги для генерации фрода.
        """
        self.atms = configs.atms
        self.client_info = None
        self.online_merchant_ids = configs.online_merchant_ids
        self.client_devices = configs.client_devices
        self.last_txn = None
    

    def original_purchase(self, online=True):
        """
        Оригинальные данные клиента для операций покупок.
        На данный момент это для дропов.
        Для операций на криптобирже и для покупки товаров дропами
        """
        if online:
            merchant_id = self.online_merchant_ids.sample(n=1).iat[0]
            # Координаты города и название
            trans_lat = self.client_info.lat
            trans_lon = self.client_info.lon
            trans_ip = self.client_info.home_ip
            trans_city = self.client_info.area        
            # Семпл девайса клиента
            devices = self.client_devices.loc[self.client_devices.client_id == self.client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            txn_type = "purchase"
            # Не генерируем channel. Он должен быть определен вовне
            channel = None

        return merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type

        
    def original_data(self, online, receive=None):
        """
        Пока этот метод для клиентов дропов и, возможно, для переводов мошенникам
        ------------------------------------
        client_id - int.
        online - bool.
        received - bool.
        """
        # Входящий перевод
        if online and receive:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "transfer"
            txn_type = "inbound"
            merchant_id = np.nan
            trans_lat = np.nan
            trans_lon = np.nan
            trans_city = "not applicable"
            
            return merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type
        
        # Исходящий перевод
        elif online:
            # Для онлайна просто берется home_ip и device_id из данных клиента.
            trans_ip = self.client_info.home_ip
            devices = self.client_devices.loc[self.client_devices.client_id == self.client_info.client_id]
            device_id = devices.device_id.sample(1).iloc[0]
            channel = "transfer"
            txn_type = "outbound"  

        # Оффлайн. Снятие в банкомате
        else:
            trans_ip = "not applicable"
            device_id = np.nan
            channel = "ATM"
            txn_type = "withdrawal"
            
        merchant_id = np.nan
        # Локация транзакции просто записываем координаты и название города клиента
        trans_lat = self.client_info.lat
        trans_lon = self.client_info.lon
        trans_city = self.client_info.area

        return merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type

In [73]:
# Временный импорт

import data_generator.fraud.txndata
importlib.reload(data_generator.fraud.txndata)
from data_generator.fraud.txndata import DropTxnPartData

In [82]:
txn_part_data_test = DropTxnPartData(configs=dist_configs)

In [85]:
for client in dist_drop_clients.itertuples():
    txn_part_data_test.client_info = client
txn_part_data_test.client_info

Pandas(Index=27, client_id=484, district_id=28, birth_date='1959-09-13', sex='female', region='Челябинская', area='Магнитогорск', timezone='UTC+5', lat=53.4071891, lon=58.9791432, population=408401, home_ip='2.60.1.205', geometry=<MULTIPOLYGON (((58.897 53.472, 58.899 53.472, 58.901 53.472, 58.903 53.472,...>)

In [86]:
txn_part_data_test.original_purchase(online=True)

(np.int64(6789),
 53.4071891,
 58.9791432,
 '2.60.1.205',
 'Магнитогорск',
 np.int64(839),
 None,
 'purchase')

In [87]:
txn_part_data_test.original_data(online=True, receive=True)
# merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, txn_type

(nan, nan, nan, 'not applicable', 'not applicable', nan, 'transfer', 'inbound')

In [88]:
txn_part_data_test.original_data(online=False)

(nan,
 53.4071891,
 58.9791432,
 'not applicable',
 'Магнитогорск',
 nan,
 'ATM',
 'withdrawal')

## Объект класса `DropTxnPartData` для распределителей
- генерация части данных транзакции

In [92]:
part_txn_data_dist = DropTxnPartData(configs=dist_configs)

## Класс `DropAccountHandler`

In [484]:
class DropAccountHandler:
    """
    Генератор номеров счетов входящих/исходящих транзакций.
    Учет использованных счетов.
    """
    def __init__(self, configs: DropConfigs, client_id, used_accounts=pd.Series(name="account_id")):
        """
        configs - pd.DataFrame. Данные для создания транзакций: номера счетов клиентов и внешних счетов, и др.
        client_id - int. id текущего дропа.
        used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
        """
        self.accounts = configs.accounts
        self.outer_accounts = configs.outer_accounts
        self.client_id = client_id
        # self.account = 0
        self.account = configs.accounts.loc[configs.accounts.client_id == client_id, "account_id"].iat[0]
        self.used_accounts = used_accounts

    def get_account(self, to_drop):
        """
        Номер счета входящего/исходящего перевода
        to_drop - bool. Перевод другому дропу в нашем банке или нет.
        """
        # Если отправляем/получаем из другого банка.  
        if not to_drop:
            # Семплируем номер внешнего счета который еще не использовался
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            # Добавляем этот счет в использованные как последнюю запись в серии
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account
        
        # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
        # Фильтруем accounts исключая свой счет и использованные счета, и выбирая дропов. Для случая если to_drop
        drop_accounts = self.accounts.loc[(self.accounts.client_id != self.client_id) & (self.accounts.is_drop == True) \
                                          & ~(self.accounts.account_id.isin(self.used_accounts))]
        
        # Если счетов дропов ещё нет. Берем внешний неиспользованный счет
        if drop_accounts.empty:
            account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
            self.used_accounts.loc[self.used_accounts.shape[0]] = account
            return account

        # Дропы есть
        account = drop_accounts.account_id.sample(1).iat[0]
        # Добавляем этот счет в использованные как последнюю запись в серии
        self.used_accounts.loc[self.used_accounts.shape[0]] = account
        return account


    def reset_cache(self):
        """
        Сброс кэшированных значений.
        Это только использованные номера счетов
        """
        self.used_accounts = pd.Series(name="account_id")

**Тест `DropAccountHandler`**

In [625]:
# импорт на время разработки

import data_generator.fraud.drops.base
importlib.reload(data_generator.fraud.drops.base)
from data_generator.fraud.drops.base import DropAccountHandler

In [626]:
drop_acc_hand1 = DropAccountHandler(configs=drop_configs)

In [627]:
drop_acc_hand1.client_id = 1

In [557]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.account

np.int64(10000)

In [520]:
drop_acc_hand1.get_account()

np.int64(18331)

In [521]:
drop_acc_hand1.used_accounts

0    18331
Name: account_id, dtype: int64

In [541]:
drop_acc_hand1.label_drop()

In [544]:
drop_acc_hand1.accounts.head(2)

,client_id,account_id,is_drop
0,1,10000,True
1,2,10001,False


In [545]:
drop_acc_hand1.client_id = 99

In [547]:
drop_acc_hand1.get_account(own=True)
drop_acc_hand1.label_drop()

In [549]:
drop_acc_hand1.accounts.query("client_id == 99")

,client_id,account_id,is_drop
93,99,10093,True


In [550]:
drop_acc_hand1.get_account(to_drop=True)

np.int64(10000)

In [551]:
drop_acc_hand1.used_accounts

0    18287
1    10000
Name: account_id, dtype: int64

In [553]:
drop_acc_hand1.reset_cache()
drop_acc_hand1.used_accounts

Series([], Name: account_id, dtype: object)

In [558]:
nodrops_acc = drop_acc_hand1.get_account(to_drop=True)
drop_acc_hand1.accounts.loc[drop_acc_hand1.accounts.account_id == nodrops_acc]

,client_id,account_id,is_drop


In [559]:
drop_acc_hand1.used_accounts.loc[drop_acc_hand1.used_accounts == nodrops_acc]

0    23419
Name: account_id, dtype: int64

## Класс `DropAmountHandler`

In [ ]:
# class DropAmountHandler: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий.
#     Управление балансом текущего дропа.
#     -------------
#     Атрибуты:
#     balance: float, int. Текущий баланс дропа. По умолчанию 0.
#     batch_txns: int. Счетчик транзакций сделанных в рамках распределения полученной партии денег.
#                       По умолчанию 0.
#     chunk_size: int, float. Последний созданный размер части баланса для перевода по частям
#                              По умолчанию 0.
#     chunks: dict. Содержит ключи:
#         atm_min: int. Минимальная сумма для снятий в банкомате.
#         atm_share: float. Доля от баланса, которую дроп снимает в случае снятия в банкомате
#         low: int. Минимальная сумма исходящего перевода частями.
#         high: int. Максимальная сумма исходящего перевода частями.
#         step: int. Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
#         rand_rate: float. От 0 до 1.
#                    Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                    Но возможны случайные совпадаения с предыдущим размером чанка.
#                    Доля случайных размеров подряд будет:
#                    p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                    p(r) - rand_rate
#                    p(r) - (p(r) * p(c)). Например p(r) = 0.9; и 5 вариантов размеров чанка - p(c) = 0.20
#                    0.9 - (0.9 * 0.2) = 0.72
#                    В около 72% случаев размеры чанков не будут подряд одинаковыми 
#     inbound_amt: dict. Настройки для сумм входящих транзакций. Содержит ключи:
#         low: int
#         high: int
#         mean: int
#         std: int
#     round: int. Округление целой части сумм транзакций. Напр. 500 значит что суммы будут кратны 500 - кончаться на 500 или 000                  
#     """

#     def __init__(self, configs: DropDistributorCfg | DropPurchaserCfg):
#         """
#         configs: DropDistributorCfg | DropPurchaserCfg. Данные на основании, которых генерируются транзакции.
#                  Отсюда берутся: atm_min, atm_share, min, step, rand_rate.
#         """
#         self.balance = 0
#         self.batch_txns = 0
#         self.chunk_size = 0
#         self.chunks = configs.chunks.copy()
#         self.inbound_amt = configs.inbound_amt.copy()
#         self.round = configs.round
#         # self.atm_min = configs.chunks["atm_min"]
#         # self.atm_share = configs.chunks["atm_share"]
#         # self.min = configs.chunks["min"]
#         # self.max = configs.chunks["max"]
#         # self.step = configs.chunks["step"]
#         # self.rand_rate = configs.chunks["rand_rate"]

#     def update_balance(self, amount, receive=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         receive - bool. Входящая ли транзакция. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if receive:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         """
#         low = self.inbound_amt["low"]
#         high = self.inbound_amt["high"]
#         mean = self.inbound_amt["mean"]
#         std = self.inbound_amt["std"]

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // self.round * self.round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, receive=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля self.chunks["atm_share"] от баланса, 
#                  но не меньше self.chunks["atm_min"]
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         rand_rate = self.chunks["rand_rate"]
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если снятие
#         if not online:
#             atm_min = self.chunks["atm_min"]
#             atm_share = self.chunks["atm_share"]
#             self.chunk_size = max(atm_min, self.balance * atm_share // self.round * self.round)
#             return self.chunk_size
        
#         # Если перевод. 
#         # Берем лимиты под генерацию массива чанков, в зависимости от
#         # полученной дропом суммы
#         small = self.chunks["rcvd_small"]
#         medium = self.chunks["rcvd_medium"] 
#         large = self.chunks["rcvd_large"]
#         step = self.chunks["step"]

#         if self.balance <= small["limit"]:
#             low = min(self.balance, small["min"]) # но не больше суммы на балансе
#             high = min(self.balance, small["max"]) # но не больше суммы на балансе

#         elif self.balance <= medium["limit"]:
#             low = min(self.balance, medium["min"])
#             high = min(self.balance, medium["max"])

#         else:
#             low = min(self.balance, large["min"])
#             high = min(self.balance, large["max"])
   
#         # прибавим шаг к максимуму, чтобы было понятнее передавать аргументы в конфиге 
#         # и не учитывать исключение значения stop в np.arange
#         sampling_array = np.arange(low, high + step, step)
#         # Если чанк больше бал
#         self.chunk_size = np.random.choice(sampling_array)
#         return self.chunk_size

**Тест `DropAmountHandler`**

In [201]:
# импорт на время разработки

import data_generator.fraud.drops.base
importlib.reload(data_generator.fraud.drops.base)
from data_generator.fraud.drops.base import DropAmountHandler
drop_amt1 = DropAmountHandler(configs=dist_configs)

In [197]:
drop_amt1.chunks

{'atm_min': 10000,
 'atm_share': 0.5,
 'step': 1000,
 'rand_rate': 0.9,
 'rcvd_small': {'limit': 10000, 'min': 3000, 'max': 10000},
 'rcvd_medium': {'limit': 30000, 'min': 5000, 'max': 25000},
 'rcvd_large': {'min': 10000, 'max': 40000}}

In [97]:
drop_amt1.balance, drop_amt1.batch_txns, drop_amt1.chunk_size

(0, 0, 0)

In [606]:
# drop_amt1.chunks, drop_amt1.receive, drop_amt1.round

In [650]:
drop_amt1.reset_cache()
drop_amt1.balance

0

In [98]:
drop_amt1.inbound_amt

{'low': 5000, 'high': 100000, 'mean': 30000, 'std': 20000}

In [99]:
drop_amt1.reset_cache()
received_list = [drop_amt1.receive(declined=True) for _ in range(10000)]
rcvd_amts_df = pd.DataFrame(pd.Series(received_list, name="rcvd_amt"))
print(f"balance: {drop_amt1.balance}")
rcvd_amts_df.rcvd_amt.agg(["min","max", "mean"])

balance: 0


min      5000.0
max     99500.0
mean    34076.9
Name: rcvd_amt, dtype: float64

In [100]:
drop_amt1.receive(declined=False)

np.float64(30500.0)

In [101]:
drop_amt1.balance

np.float64(30500.0)

In [102]:
drop_amt1.receive(declined=False)

np.float64(28000.0)

In [103]:
drop_amt1.balance

np.float64(58500.0)

In [104]:
print("before: ", drop_amt1.balance)
drop_amt1.update_balance(amount=7000, receive=False, declined=True)
drop_amt1.balance

before:  58500.0


np.float64(58500.0)

**Тест `chunk_size`**

In [689]:
# Значения атрибутов отличные от прописанных в конфиге - закомментить/раскомментить при надобности
# drop_amt1.chunks["low"] = 10000
# drop_amt1.chunks["high"] = 45000
# drop_amt1.chunks["step"] = 1000
# drop_amt1.chunks["atm_min"] = 15000
# drop_amt1.chunks["atm_share"] = 0.4
# drop_amt1.chunks["rand_rate"] = 0.5

In [204]:
drop_amt1.balance = 10000
drop_amt1.batch_txns = 1
drop_amt1.get_chunk_size(online=False), drop_amt1.chunk_size

(10000, 10000)

In [761]:
drop_amt1.batch_txns = 1
# drop_amt1.chunk_size = 20000
drop_amt1.balance = drop_amt1.chunks["rcvd_small"]["limit"]
chunks_list = [drop_amt1.get_chunk_size(online=True) for _ in range(1000)]
chunks_df = pd.DataFrame(pd.Series(chunks_list, name="chunk_size"))
chunks_df.head()

,chunk_size
0,7000
1,4000
2,4000
3,10000
4,8000


In [762]:
chunks_df["prev_size"] = chunks_df.chunk_size.shift(1)
chunks_df["eq_to_prev"] = chunks_df["chunk_size"] == chunks_df["prev_size"]
chunks_df.head(3)

,chunk_size,prev_size,eq_to_prev
0,7000,NaN,False
1,4000,7000.0,False
2,4000,4000.0,True


In [105]:
rand_rate1 = drop_amt1.chunks["rand_rate"]

# Берем лимиты под генерацию массива чанков, в зависимости от
# полученной дропом суммы
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

if drop_amt1.balance <= small["limit"]:
    low = small["min"]
    high = min(drop_amt1.balance, small["max"])

elif drop_amt1.balance <= medium["limit"]:
    low = medium["min"]
    high = min(drop_amt1.balance, medium["max"])

else:
    low = large["min"]
    high = min(drop_amt1.balance, large["max"])
    
step = drop_amt1.chunks["step"]
chunks_num = np.arange(start=low, stop=high + step, step=step).shape[0]
one_chunk_p = 1 / chunks_num
print(f"""balance: {drop_amt1.balance}
rand_rate: {rand_rate1}
chunks number: {chunks_num}
prob of taking one chunk: {one_chunk_p:.3f}
{rand_rate1 - (rand_rate1 * one_chunk_p):.3f} chunks won't be the same as a previous chunk size""")

balance: 58500.0
rand_rate: 0.9
chunks number: 31
prob of taking one chunk: 0.032
0.871 chunks won't be the same as a previous chunk size


In [764]:
chunks_df.eq_to_prev.value_counts(normalize=True)

eq_to_prev
False    0.788
True     0.212
Name: proportion, dtype: float64

In [757]:
chunks_df.chunk_size.agg(["min","max"])

min     5000
max    20000
Name: chunk_size, dtype: int64

**тест `one_operation`**

In [ ]:
# chunks: # Настройки создания суммы, для распределения полученных денег по частям
# atm_min: 10000 # Минимальная сумма для снятий в банкомате.
# atm_share: 0.5 # Доля от баланса, которую дроп снимает в случае снятия в банкомате
# step: 1000 # Шаг возможных сумм. Чем меньше шаг, тем больше вариантов.
# rand_rate: 0.9 # В скольких случаях размер суммы должен быть семплирован, а не взят размер предыдущей транзакции
# rcvd_small: # Категория полученной дропом суммы
#   limit: 10000 # Категория все что "до" включительно
#   min: 3000 # # Минимальная сумма чанка для этой категории
#   max: 10000 # Максимальная сумма чанка для этой категории
# rcvd_medium:
#   limit: 30000
#   min: 5000
#   max: 25000
# rcvd_large: # порог не прописываем, т.к. это все что выше medium
#   min: 10000
#   max: 40000

In [109]:
small = drop_amt1.chunks["rcvd_small"]
medium = drop_amt1.chunks["rcvd_medium"] 
large = drop_amt1.chunks["rcvd_large"]

**Ниже ячейка для кейсов:**
- Не отклонена. Онлайн. Частями. Small
- Не отклонена. Онлайн. Частями. Medium
- Не отклонена. Онлайн. Частями. Large

- `min_lim` всегда стоит `min_lim = small["min"]` т.к. баланс уменьшается и могут срабатывать условия для `small`, `medium`
- `drop_amt1.balance` для large ставить как `medium["limit"] + число` т.к. у large нет своего лимита, используется все что выше `medium["limit"]`

In [225]:
i = 0
min_lim = small["min"]
max_lim = small["max"]

while i < 1000:
    drop_amt1.reset_cache()
    drop_amt1.balance = small["limit"]
    all_ops = []
    
    while drop_amt1.balance > 0:
        one_op1 = drop_amt1.one_operation(online=True, declined=False, in_chunks=True)
        all_ops.append(one_op1)
        i += 1
    all_ops_ser = pd.Series(all_ops)
    # для минимального берем серию без последнего элемента. Т.к. там обычно уже остаток, а не чанк
    if all_ops_ser.shape[0] == 1:
        assert all_ops_ser.min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.min()}"
    else:
        assert all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min() >= min_lim, \
                            f"Min is lower than min limit. {all_ops_ser.iloc[0:(all_ops_ser.shape[0] - 1)].min()}"
        
    assert all_ops_ser.max() <= max_lim, "Max is higher than min limit."

In [195]:
all_ops_ser

0    12000
1    15000
2     3000
3     1000
dtype: int64

In [227]:
atm_min = drop_amt1.chunks["atm_min"]

In [232]:
drop_amt1.reset_cache()
drop_amt1.balance = atm_min - 1000
print(drop_amt1.balance)
all_ops2 = []

while drop_amt1.balance > 0:
    one_op2 = drop_amt1.one_operation(online=False, declined=False, in_chunks=True)
    all_ops2.append(one_op2)
all_ops_ser = pd.Series(all_ops2)
all_ops_ser

9000


0    9000
dtype: int64

**Тест отклоненных транзакций. Онлайн/оффлайн**

In [234]:
drop_amt1.reset_cache()
drop_amt1.balance = atm_min + 37000
print(drop_amt1.balance)
one_op_list1 = []
while True:
    one_op1 = drop_amt1.one_operation(online=False, declined=True, in_chunks=True)
    one_op_list1.append(one_op1)
    if len(one_op_list1) == 5:
        break
print(drop_amt1.balance, "\n", one_op_list1)

47000
47000 
 [23500.0, 23500.0, 23500.0, 23500.0, 23500.0]


## Класс `DropTimeHandler`
- управление временем транзакций дропа

In [238]:
# <------------------------- ГОТОВЫЙ КЛАСС ВСТАВИТЬ--------------------------

**Тест `DropTimeHandler`**

In [239]:
# Временный reload для разработки
import data_generator.configs
importlib.reload(data_generator.configs)
import data_generator.fraud.drops.time
importlib.reload(data_generator.fraud.drops.time)
from data_generator.configs import DropDistributorCfg, DropPurchaserCfg
from data_generator.fraud.drops.time import DropTimeHandler

drop_time_hanler1 = DropTimeHandler(configs=dist_configs)

In [240]:
two_way_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=True) for _ in range(1000)])
two_way_deltas1.mean(), two_way_deltas1.min(), two_way_deltas1.max()

(np.float64(-2.687), np.int64(-180), np.int64(180))

In [241]:
pos_deltas1 = pd.Series([drop_time_hanler1.get_time_delta(two_way=False) for _ in range(1000)])
pos_deltas1.mean(), pos_deltas1.min(), pos_deltas1.max()

(np.float64(104.085), np.int64(30), np.int64(180))

In [242]:
for _ in range(3):
    drop_time_hanler1.txns_count(receive=True, reset=False)
    drop_time_hanler1.txns_count(receive=False, reset=False)

In [243]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(3, 3)

In [393]:
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

In [245]:
drop_time_hanler1.in_txns = 2
drop_time_hanler1.out_txns = 4
drop_time_hanler1.txns_count(receive=True, reset=True)
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(1, 0)

**`DropTimeHandler.get_txn_time` тест**  
Тест-кейсы
- Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции
- Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- Не первая транзакция и достигнут лимит исходящих транзакций для периода активности
- Не первая транзакция. Лимиты не достигнуты. Транзакция входящая
- Не первая транзакция. Лимиты не достигнуты. Транзакция исходящая 

In [246]:
start_time = pd.to_datetime("2025-07-02 11:15:00", format="%Y-%m-%d %H:%M:%S")
start_unix = pd_timestamp_to_unix(start_time)
start_time, start_unix

(Timestamp('2025-07-02 11:15:00'), 1751454900)

In [247]:
last_time = pd.to_datetime("2025-07-02 18:31:00", format="%Y-%m-%d %H:%M:%S")
last_unix = pd_timestamp_to_unix(last_time)
last_time, last_unix

(Timestamp('2025-07-02 18:31:00'), 1751481060)

1. Если это самая первая транзакция. Т.к. активность дропа начинается с входящей транзакции

In [248]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = 0
drop_time_hanler1.out_txns = 0
droptime1, dropunix1 = drop_time_hanler1.get_txn_time(receive=True, in_txns=0)
droptime1, dropunix1

(Timestamp('2025-01-05 13:47:00'), np.int64(1736084820))

In [249]:
drop_time_hanler1.last_unix, drop_time_hanler1.in_txns, drop_time_hanler1.out_txns

(np.int64(1736084820), 1, 0)

2. Не первая транзакция и достигнут лимит входящих транзакций для периода активности
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [250]:
start_time, last_time

(Timestamp('2025-07-02 11:15:00'), Timestamp('2025-07-02 18:31:00'))

In [254]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime2, dropunix2 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start2 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime2, dropunix2, droptime_start2

(Timestamp('2025-07-03 12:05:00'),
 1751544300,
 Timestamp('2025-07-03 12:05:00'))

In [255]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 0, 1751544300, 1751544300)

In [256]:
droptime2 - start_time

Timedelta('1 days 00:50:00')

3. Не первая транзакция и достигнут лимит исходящих транзакций для периода активности  
- текущая входящая - receive=True
- текущая исходящая - receive=False

In [260]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime3, dropunix3 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start3 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime3, dropunix3, droptime_start3

(Timestamp('2025-07-03 12:56:00'),
 1751547360,
 Timestamp('2025-07-03 12:56:00'))

In [261]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(0, 1, 1751547360, 1751547360)

In [262]:
droptime3 - start_time

Timedelta('1 days 01:41:00')

4. Не первая транзакция. Лимиты не достигнуты. Транзакция **входящая**

In [263]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime4, dropunix4 = drop_time_hanler1.get_txn_time(receive=True, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start4 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime4, dropunix4, droptime_start4

In txns: 1 
Out txns: 4
120


(Timestamp('2025-07-02 20:31:00'),
 1751488260,
 Timestamp('2025-07-02 11:15:00'))

In [264]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(2, 4, 1751454900, 1751488260)

In [265]:
droptime4 - last_time

Timedelta('0 days 02:00:00')

5. Не первая транзакция. Лимиты не достигнуты. Транзакция **исходящая**

In [266]:
drop_time_hanler1.reset_cache()
drop_time_hanler1.in_txns = drop_time_hanler1.in_lim - 1
drop_time_hanler1.out_txns = drop_time_hanler1.out_lim - 1
print(f"In txns: {drop_time_hanler1.in_txns} \nOut txns: {drop_time_hanler1.out_txns}")
drop_time_hanler1.start_unix = start_unix
drop_time_hanler1.last_unix = last_unix
droptime5, dropunix5 = drop_time_hanler1.get_txn_time(receive=False, in_txns=drop_time_hanler1.in_lim * 2)
droptime_start5 = pd.to_datetime(drop_time_hanler1.start_unix, unit="s")
droptime5, dropunix5, droptime_start5

In txns: 1 
Out txns: 4
179


(Timestamp('2025-07-02 21:30:00'),
 1751491800,
 Timestamp('2025-07-02 11:15:00'))

In [267]:
drop_time_hanler1.in_txns, drop_time_hanler1.out_txns, drop_time_hanler1.start_unix, drop_time_hanler1.last_unix

(1, 5, 1751454900, 1751491800)

In [268]:
droptime5 - last_time

Timedelta('0 days 02:59:00')

## Класс `DistBehaviorHandler`

In [ ]:
# ---------------------------- ВСТАВИТЬ КЛАСС ----------------------------------

In [269]:
limits1 = {"transfer":{"trf_lim":"max"}, "atm":{"atm_min":"min"}, \
         "split_transfer":{"trf_min":"min"}, "atm+transfer":{"atm_min":"min", "trf_min":"min"}}

In [272]:
lims_cfg1 = {"trf_lim":40000, "atm_min":10000, "trf_min":3000*2}

In [ ]:
for key in limits1.keys():
    temp_lim1 = []
    for nest_key in limits1[key].keys():
        if limits1[key][nest_key] == "min":

In [ ]:
# inbound_amt: 
#   low: 5000
#   high: 100000
#   mean: 30000
#   std: 20000

In [308]:
# Баланс генерируется из обрезанного нормального распределения
in_low1 = 5000
in_high1 = 100000
mean1 = 30000
std1 = 20000
a1 = (in_low1 - mean1) / std1
b1 = (in_high1 - mean1) / std1

split_rate = 0.7
# Есть лимиты для сумм операций
atm_min1 = 10000 # минимальная сумма для снятия. Частями или целиком определяется split_rate
trf_lim = 40000 # максимальная сумма для перевода. Если больше - дробим на части
trf_min = 3000*2 # минимальная сумма для перевода по частям. Если меньше - переводим целиком

In [309]:
# вероятность что сумма будет больше trf_lim
p_trf_lim = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_lim), 3)
p_trf_lim

np.float64(0.345)

In [310]:
# Вероятность что сумма больше atm_min1 но меньше trf_lim
p_atm_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=atm_min1) - p_trf_lim, 3)
p_atm_min

np.float64(0.596)

In [311]:
# вероятность что сумма больше trf_min, но меньше trf_lim и atm_min1
p_trf_min = round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min) - p_trf_lim - p_atm_min, 3)
p_trf_min

np.float64(0.048)

In [312]:
split_rate = 0.7 # доля когда будем переводить частями

(p_trf_lim + p_atm_min + p_trf_min) * split_rate

np.float64(0.6922999999999999)

In [313]:
round(truncnorm(a=a1, b=b1, loc=mean1, scale=std1).sf(x=trf_min), 3) * split_rate

np.float64(0.6922999999999999)

In [315]:

# conditions = [
#             (balance > self.trf_lim, ["atm","split_transfer", "atm+transfer"]),
#             (balance >= self.atm_min, ["transfer", "atm", "split_transfer", "atm+transfer"]),
#             (balance >= self.trf_min * 2, ["transfer","split_transfer"]),
#             "transfer",
#         ]
# Вероятность случаев перевода частями если по условиям выше (синтаксически неправильно) и без split_rate
p_trf_lim * 2/3 + p_atm_min * 1/2 + p_trf_min * 1/2

np.float64(0.552)

In [ ]:
def sample_scenario(self):
    """
    Выбор сценария поведения дропа с учётом текущего баланса и актуальных лимитов.
    """
    balance = self.amt_hand.balance
    split = False
    if np.random.uniform(0,1) <= self.split_rate:
            split = True
            
    # Список: (условие, список возможных сценариев)
    # Минимальный баланс для переводов по частям будет самый маленький возможный размер чанка
    # self.trf_min умноженный на 2
    conditions = [
                (balance > self.trf_lim and split, ["split_transfer", "atm+transfer"]),
                (balance > self.trf_lim, ["atm"]),
                (balance >= self.atm_min and split, ["split_transfer", "atm+transfer"]),
                (balance >= self.atm_min, ["transfer", "atm"]),
                (balance >= self.trf_min * 2 and split, ["split_transfer"]),
                ]

    for cond, scen_list in conditions:
        if cond:
            return np.random.choice(scen_list)

    # Если ни одно из условий не сработало — fallback
    return "transfer"

## Класс `CreateDropTxn`

In [270]:
class CreateDropTxn:
    """
    Создание транзакций дропа под разное поведение.
    """
    def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
                attempts=0):
        """
        timestamps - pd.DataFrame.
        trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
        drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
        in_txns - int. Количество входящих транзакций
        out_txns - int. Количество исходящих транзакций
        in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
        last_txn - dict. Полные данные последней транзакции
        attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
        """
        self.timestamps = timestamps
        self.trans_partial_data = trans_partial_data
        self.drop_client = drop_client_cls
        self.in_txns = in_txns
        self.out_txns = out_txns
        self.in_lim = in_lim
        self.out_lim = out_lim
        self.last_txn = last_txn
        self.attempts = attempts


    def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False): # <------------------------- KEEP, EDIT
        """
        Один входящий/исходящий перевод либо одно снятие в банкомате.
        ---------------------
        online - bool. Онлайн перевод или снятие в банкомате.
        declined - bool. Будет ли текущая транзакция отклонена.
        in_chunks - bool. Транзакция будет частью серии транзакций.
        to_drop_share - float. Вероятность, что дроп пошлет другому дропу
        receive - входящий перевод или нет.
        """
        client_id = self.trans_partial_data.client_info.client_id # берем из namedtuple
        
        # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
        txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

        # перевод дропу
        if receive:
            self.in_txns += 1
            amount = self.drop_client.receive(declined=declined)
            account = self.drop_client.account
        # перевод от дропа    
        elif not receive and online:
            to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
            self.out_txns += 1
            account = self.drop_client.get_account(to_drop=to_drop)
        # снятие дропом    
        elif not receive and not online:
            account = self.drop_client.account
            self.out_txns += 1
        
        # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
        merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
                                                self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
        # Генерация суммы если исходящая транзакция
        # Если эта транзакция только часть серии операций для распределения всего баланса
        if in_chunks:
            chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)
            
        # Иначе если не по частям и не входящая транзакция. not receive т.к. этот метод и для входящих транзакций
        # а у входящих транзакций своя генерация суммы
        elif not in_chunks and not receive:
            amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

        if declined:
            status = "declined"
            is_fraud = True
            rule = "drop_flow_cashout"
        else:
            status = "approved"
            is_fraud = False
            rule = "not applicable"

        # Статичные характеристики
        is_suspicious = False
        category_name="not applicable"

        # Сборка всех данных в транзакцию и запись как послдней транзакции
        self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
                             category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
                             trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
                             is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

        return self.last_txn

    
    def limit_reached(self): # <------------------------- KEEP
        """
        Проверка достижения лимитов входящих и исходящих транзакций
        Сверка с self.in_lim и self.out_lim
        ------------------------
        Вернет True если какой либо лимит достигнут
        """
        if self.in_lim == self.in_txns:
            return True
        if self.out_lim == self.out_txns:
            return True
        return False

    def reset_cache(self, only_counters=True): # <------------------------- KEEP, EDIT
        """
        Сброос кэшированных данных
        -------------
        only_counters - bool
                        Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
                        Если False то также сбросится информация о последней транзакции self.last_txn
        """
        
        self.in_txns = 0
        self.out_txns = 0
        self.batch_txns = 0

        if only_counters:
            return

        self.last_txn = {}

In [ ]:
# class CreateDropTxn:
#     """
#     Создание транзакций дропа под разное поведение.
#     """
#     def __init__(self, timestamps, trans_partial_data, drop_client_cls, in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, \
#                 attempts=0):
#         """
#         timestamps - pd.DataFrame.
#         trans_partial_data - FraudTransPartialData. Генератор части данных транзакции - мерчант, гео, ip, девайс и т.п.
#         drop_client_cls - DropClient. Генератор активности дропов: суммы, счета, баланс
#         in_txns - int. Количество входящих транзакций
#         out_txns - int. Количество исходящих транзакций
#         in_lim - int. Лимит входящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
#         out_lim - int. Лимит исходящих транзакций. Транзакции клиента совершенные после достижения этого лимита отклоняются
#         last_txn - dict. Полные данные последней транзакции
#         attempts - int. Сколько попыток совершить операцию будет сделано дропом после первой отклоненной транзакции.
#         """
#         self.timestamps = timestamps
#         self.trans_partial_data = trans_partial_data
#         self.drop_client = drop_client_cls
#         self.in_txns = in_txns
#         self.out_txns = out_txns
#         self.in_lim = in_lim
#         self.out_lim = out_lim
#         self.last_txn = last_txn
#         self.attempts = attempts

#     def get_time_delta(self, min=-180, max=180, minutes=True):
#         """
#         Получение случайного интервала времени в секундах или минутах из равномерного распределения
#         ---------------------
#         min - int. Минимальное возможное значение
#         max - int. Максимальное возможное значение
#         minutes - bool. Минуты или секунды
#         """
#         if minutes:
#             return round(np.random.uniform(min, max))
            
#         return round(np.random.uniform(min, max) * 60)

#     def get_txn_time(self, in_lim=2, out_lim=5, lag_interval=1440):
#         """
#         Генерация времени транзакции
#         ------------------
#         in_lim - int. Количество входящих транзакций после которых дроп уходит на паузу указанную в lag_interval
#                       Т.е. если на момент генерации времени уже сделано in_lim транзакций, то берется время последней
#                       транзакции и прибавляется указанный lag_interval +/- случайное число минут из delta.
#         out_lim - int. Количество исходящих транзакций после которых дроп уходит на паузу указанную в lag_interval
#         lag_interval - int. Желаемый лаг по времени от последней транзакции в минутах.
#                             Используется для перерывов в активности дропа. По умолчанию 1440 минут т.е. 24 часа
#         """
#         # Если это первая транзакция. Т.к. активность дропа начинается с входящей транзакции
#         if self.in_txns == 0:
#             time_sample = self.timestamps.sample(1)
#             txn_time = time_sample.timestamp.iat[0]
#             txn_unix = time_sample.unix_time.iat[0]
#             return txn_time, txn_unix

#         # Для последующих транзакций
#         last_txn_unix = self.last_txn["unix_time"]

#         # Если достигнуты лимиты активности дропа на период: входящих или исходящих транзакций
#         if self.in_txns == in_lim or self.out_txns == out_lim:
#             # Генерация дельты, чтобы время выглядело не ровным, а случайным.
#             # Слагаем её с lag_interval
#             time_delta = self.get_time_delta(min=-180, max=180)
#             lag_interval += time_delta
#             return derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=lag_interval)

#         # Тоже дельта, но не может быть <= 0 т.к. тут мы ее используем как lag_interval
#         # Это для случаев когда транзакция совершается в тот же период активности что и последняя
#         time_delta = self.get_time_delta(min=30, max=180)
        

#         return derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=time_delta)

    
#     def stop_after_decline(self, declined):
#         """
#         Будет ли дроп пытаться еще после отклоненной операции
#         или остановится
#         ---------------
#         declined - bool. Отклонена ли операция. Подразумевается что последняя.
#         """
#         if not declined:
#             return
        
#         if self.attempts == 0:
#             return True

#         if self.attempts > 0:
#             return False

            
#     def attempts_after_decline(self, min=0, max=4):
#         """
#         Определение количества попыток после первой отклоненной транзакции
#         ---------------
#         min - int. Минимальное число попыток
#         max - int. Максимальное число попыток.
#         """
#         self.attempts = np.random.randint(min, max + 1)
            
        
#     def deduct_attempts(self, declined, receive):
#         """
#         Вычитание попытки операции совершенной при статусе declined
#         ---------------
#         declined - bool. Отклоняется ли текущая транзакция
#         receive - bool. Является ли транзакция входящей
#         """
#         if self.attempts == 0:
#             return 
            
#         if declined and not receive:
#             self.attempts -= 1
            
        
#     def cash_flow_action(self, online, declined, in_chunks, to_drop_share=0.2, receive=False):
#         """
#         Один входящий/исходящий перевод либо одно снятие в банкомате.
#         ---------------------
#         online - bool. Онлайн перевод или снятие в банкомате.
#         declined - bool. Будет ли текущая транзакция отклонена.
#         in_chunks - bool. Транзакция будет частью серии транзакций.
#         to_drop_share - float. Вероятность, что дроп пошлет другому дропу
#         receive - входящий перевод или нет.
#         """
#         client_id = self.trans_partial_data.client_info.client_id
        
#         # Время транзакции. Оно должно быть создано до увеличения счетчика self.in_txns
#         txn_time, txn_unix = self.get_txn_time(in_lim=2, out_lim=5, lag_interval=1440)

#         if receive:
#             self.in_txns += 1
#             amount = self.drop_client.receive(declined=declined)
#             account = self.drop_client.account
            
#         elif not receive and online:
#             to_drop = np.random.choice([True, False], p=[to_drop_share, 1 - to_drop_share])
#             self.out_txns += 1
#             account = self.drop_client.get_account(to_drop=to_drop)
            
#         elif not receive and not online:
#             account = self.drop_client.account
#             self.out_txns += 1
        
            
#         # Генерация части данных транзакции. Здесь прописываются аргументы online и receive
#         merchant_id, trans_lat, trans_lon, trans_ip, trans_city, device_id, channel, type = \
#                                                 self.trans_partial_data.original_transfer_or_atm(online=online, receive=receive)
        
#         # Генерация суммы если исходящая транзакция
#         # Если эта транзакция только часть серии операций для распределения всего баланса
#         if in_chunks:
#             chunk = self.drop_client.get_chunk_size(online=online, atm_min=10000, start=5000, stop=25000, step=5000)
#             amount = self.drop_client.one_operation(amount=chunk, declined=declined, in_chunks=in_chunks)
            
#         # Иначе если не по частям и не входящая транзакция
#         elif not in_chunks and not receive:
#             amount = self.drop_client.one_operation(declined=declined, in_chunks=in_chunks)

#         if declined:
#             status = "declined"
#             is_fraud = True
#             rule = "drop_flow_cashout"
#         else:
#             status = "approved"
#             is_fraud = False
#             rule = "not applicable"

#         # Статичные характеристики
#         is_suspicious = False
#         category_name="not applicable"

#         # Сборка всех данных в транзакцию и запись как послдней транзакции
#         self.last_txn = build_transaction(client_id=client_id, txn_time=txn_time, txn_unix=txn_unix, amount=amount, type=type, channel=channel, \
#                              category_name=category_name, online=online, merchant_id=merchant_id, trans_city=trans_city, \
#                              trans_lat=trans_lat, trans_lon=trans_lon, trans_ip=trans_ip, device_id=device_id, account=account, \
#                              is_fraud=is_fraud, is_suspicious=is_suspicious, status=status, rule=rule)

#         return self.last_txn

#     def limit_reached(self):
#         """
#         Проверка достижения лимитов входящих и исходящих транзакций
#         Сверка с self.in_lim и self.out_lim
#         ------------------------
#         Вернет True если какой либо лимит достигнут
#         """
#         if self.in_lim == self.in_txns:
#             return True
#         if self.out_lim == self.out_txns:
#             return True
#         return False

#     def reset_txn_counters(self, in_txns=False, out_txns=False, batch_txns=False):
#         """
#         Сброс счетчиков входящих и/или исходящих транзакций
#         Партия денег это полученные деньги которые дроп должен распределить
#         ------------------
#         in_txns - bool. Сбросить счетчик входящих.
#         out_txns - bool. Сбросить счетчик исходящих.
#         batch_txns - bool. Сбросить счетчик транзакций партии денег.
#         """
#         if in_txns:
#             self.in_txns = 0
#         if out_txns:
#             self.out_txns = 0
#         if batch_txns:
#             self.batch_txns = 0

#     def reset_cache(self, only_counters=True):
#         """
#         Сброос кэшированных данных
#         -------------
#         only_counters - bool
#                         Если True будут сброшены: self.in_txns, self.out_txns, self.attempts.
#                         Если False то также сбросится информация о последней транзакции self.last_txn
#         """
        
#         self.in_txns = 0
#         self.out_txns = 0
#         self.attempts = 0

#         if only_counters:
#             return

#         self.last_txn = {}

**`CreateDropTxn.single_operation` тест**

In [159]:
drop_stamps = create_timestamps_range_df(start="2025-01-01", end="2025-01-31")

In [260]:
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)

In [261]:
# drop_txn_part_data.client_info

In [262]:
drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

In [263]:
create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)

In [264]:
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, create_drop_txn_tst.last_txn, drop_client_test2.batch_txns

(0, 0, {}, 0)

In [1342]:
# ?CreateDropTxn.single_operation

Входящая НЕотклоненная транзакция дропу

In [205]:
create_drop_txn_tst.reset_cache(only_counters=False)
receive_txn = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
pd.DataFrame([receive_txn])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-02 12:17:00,1735820220,52000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable


In [206]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(52000.0), 1)

In [207]:
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, create_drop_txn_tst.attempts

(1, 0, 0)

In [1542]:
# create_drop_txn_tst.last_txn

НЕ отклоненный исходящий перевод целиком

In [208]:
whole_out = create_drop_txn_tst.single_operation(online=True, receive=False, declined=False, in_chunks=False)
pd.concat([pd.DataFrame([receive_txn]), pd.DataFrame([whole_out])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-02 12:17:00,1735820220,52000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-02 13:25:00,1735824300,52000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,21193,False,False,approved,not applicable


НЕ отклоненное снятие целиком

In [209]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn2 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
whole_atm = create_drop_txn_tst.single_operation(online=False, receive=False, declined=False, in_chunks=False)

pd.concat([pd.DataFrame([receive_txn2]), pd.DataFrame([whole_atm])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-24 01:00:00,1737680400,6500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-24 02:35:00,1737686100,6500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable


In [210]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(0.0), 1)

НЕ отклоненный исходящий перевод частями

In [267]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn3 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
pd.DataFrame([receive_txn3])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-22 12:27:00,1737548820,36500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable


In [268]:
all_txns3 = [receive_txn3]
while drop_client_test2.balance > 0:
    part_out = create_drop_txn_tst.single_operation(online=True, receive=False, declined=False, in_chunks=True)
    all_txns3.append(part_out)
pd.DataFrame(all_txns3)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account,is_fraud,is_suspicious,status,rule
0,1,2025-01-22 12:27:00,1737548820,36500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-22 15:20:00,1737559200,20000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,16461,False,False,approved,not applicable
2,1,2025-01-22 18:11:00,1737569460,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,18060,False,False,approved,not applicable
3,1,2025-01-22 20:14:00,1737576840,11500.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,19418,False,False,approved,not applicable


In [269]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(0.0), 3)

НЕ отклоненное снятие частями

In [1700]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn4 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)

all_txns4 = [receive_txn4]
while drop_client_test2.balance > 0:
    part_out = create_drop_txn_tst.single_operation(online=False, receive=False, declined=False, in_chunks=True)
    all_txns4.append(part_out)
pd.DataFrame(all_txns4)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-17 08:20:00,1737102000,45000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-17 10:35:00,1737110100,22500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable
2,1,2025-01-17 11:40:00,1737114000,22500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,False,False,approved,not applicable


Отклоненный входящий перевод

In [1706]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn5 = create_drop_txn_tst.single_operation(online=True, receive=True, declined=True, in_chunks=False)
pd.DataFrame([receive_txn5])

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-01 07:56:00,1735718160,30000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,True,False,declined,client_is_drop


In [1707]:
drop_client_test2.balance, drop_client_test2.batch_txns

(0, 0)

Отклоненный исходящий перевод целиком

In [ ]:
online=True, receive=False, declined=True, in_chunks=False

In [1710]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn6 = create_drop_txn_tst.single_operation(online=True, receive=True, declined=False, in_chunks=False)

whole_out2 = create_drop_txn_tst.single_operation(online=True, receive=False, declined=True, in_chunks=False)


pd.concat([pd.DataFrame([receive_txn6]), pd.DataFrame([whole_out2])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-30 12:16:00,1738239360,25000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-30 14:51:00,1738248660,25000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,17922,True,False,declined,client_is_drop


In [1711]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(25000.0), 1)

**Отклоненное снятие целиком**

In [1781]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
receive_txn7 = create_drop_txn_tst.single_operation(online=True, declined=False, in_chunks=False, receive=True)
whole_atm_2 = create_drop_txn_tst.single_operation(online=False, receive=False, declined=True, in_chunks=False)

pd.concat([pd.DataFrame([receive_txn7]), pd.DataFrame([whole_atm_2])], ignore_index=True)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-03 14:40:00,1735915200,29500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-03 15:42:00,1735918920,29500.0,withdrawal,ATM,not applicable,False,NaN,Рязань,54.625457,39.735999,not applicable,<NA>,1,True,False,declined,client_is_drop


In [1782]:
drop_client_test2.balance, drop_client_test2.batch_txns

(np.float64(29500.0), 1)

**Отклоненный  исходящий перевод частями и заодно набросок полного цикла дропа распределителя**

In [1794]:
# Объявление новых объектов классов для тестов.
drop_txn_part_data = FraudTransPartialData(merchants_df=pd.DataFrame(), client_info=clients_with_geo.loc[0], \
                                        online_merchant_ids=pd.DataFrame(), fraud_ips=fraud_ips, used_ips=pd.Series(), \
                                         fraud_devices=fraud_devices,  used_devices=pd.Series(), \
                                        client_devices=client_devices)


drop_client_test2 = DropClient(accounts=accounts, account=1, outer_accounts=outer_accounts)

create_drop_txn_tst = CreateDropTxn(timestamps=drop_stamps, trans_partial_data=drop_txn_part_data, drop_client_cls=drop_client_test2, \
                                      in_txns=0, out_txns=0, in_lim=6, out_lim=8, last_txn={}, attempts=0)
create_drop_txn_tst.reset_cache(only_counters=False)
create_drop_txn_tst.in_txns, create_drop_txn_tst.out_txns, drop_client_test2.batch_txns, create_drop_txn_tst.limit_reached()

(0, 0, 0, False)

In [ ]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
all_txns5 = []
declined=False
create_drop_txn_tst.attempts_after_decline(min=0, max=1)
drop_client_test2.chunk_size = 5000
drop_client_test2.batch_txns = 1

i = 1

while True:
    receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
    all_txns5.append(receive_txn8)
    if declined:
        break
        
    while drop_client_test2.balance > 0:
        part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
        all_txns5.append(declined)
        print(f"iter {i}")
        create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
        if create_drop_txn_tst.stop_after_decline(declined=declined):
            break
           
        declined = create_drop_txn_tst.limit_reached()
        i += 1
        
    # pd.DataFrame(all_txns5)

Ниже нерабочий код т.к. это прикидки с криптой

In [1795]:
create_drop_txn_tst.reset_cache(only_counters=False)
drop_client_test2.reset_cache(balance=True, used_accounts=True, chunk_size=True, batch_txns=True)
all_txns5 = []
declined=False
crypto_rate = 0.05
create_drop_txn_tst.attempts_after_decline(min=0, max=1)
drop_client_test2.chunk_size = 5000
drop_client_test2.batch_txns = 1

i = 1

while True:
    receive_txn8 = create_drop_txn_tst.single_operation(online=True, declined=declined, in_chunks=False, receive=True)
    all_txns5.append(receive_txn8)
    if declined:
        break
        
    while drop_client_test2.balance > 0:
        if np.random.uniform(0, 1) < crypto_rate:
            # использовать метод FraudTransPartialData.original_purchase внутри
            part_out = create_drop_txn_tst.purchase(online=True, declined=declined, in_chunks=True, channel="crypto_exchange")
        else:
            part_out = create_drop_txn_tst.cash_flow_action(online=True, receive=False, declined=declined, in_chunks=True)
        all_txns5.append(part_out)
        print(f"iter {i}")
        create_drop_txn_tst.deduct_attempts(declined=declined, receive=False) 
        if create_drop_txn_tst.stop_after_decline(declined=declined):
            break
           
        declined = create_drop_txn_tst.limit_reached()
        i += 1
        
    # pd.DataFrame(all_txns5)

iter 1
iter 2
iter 3
iter 4
iter 5
iter 6
iter 7
iter 8
iter 9


In [1798]:
pd.DataFrame(all_txns5)

,client_id,txn_time,unix_time,amount,type,channel,category,online,merchant_id,trans_city,trans_lat,trans_lon,trans_ip,device_id,account_to,is_fraud,is_suspicious,status,rule
0,1,2025-01-26 19:17:00,1737919020,9000.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
1,1,2025-01-26 19:54:00,1737921240,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,17264,False,False,approved,not applicable
2,1,2025-01-26 22:17:00,1737929820,4000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,18967,False,False,approved,not applicable
3,1,2025-01-26 23:24:00,1737933840,32500.0,inbound,transfer,not applicable,True,NaN,not applicable,NaN,NaN,not applicable,<NA>,1,False,False,approved,not applicable
4,1,2025-01-27 21:51:00,1738014660,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,20290,False,False,approved,not applicable
5,1,2025-01-28 19:29:00,1738092540,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,20575,False,False,approved,not applicable
6,1,2025-01-29 18:13:00,1738174380,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,19220,False,False,approved,not applicable
7,1,2025-01-30 16:10:00,1738253400,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,23528,False,False,approved,not applicable
8,1,2025-01-31 13:16:00,1738329360,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,2,24752,False,False,approved,not applicable
9,1,2025-02-01 10:57:00,1738407420,5000.0,outbound,transfer,not applicable,True,NaN,Рязань,54.625457,39.735999,2.60.0.1,1,16527,False,False,approved,not applicable


np.float64(2500.0)

## Функция генерации транзакции дропа `gen_drop_txn`

In [1074]:
def one_batch_dist(drop_info, drop_client_act: DropClient, batch):
    """
    Распределение дропом одной партии полученных денег
    -------------------
    drop_info - namedtuples из itertuples(). Информация клиента-дропа из общего датафрейма клиентов
    drop_client_act - DropClient. Генератор атрибутов дропа и его действий; генератор счетов транзакций дропа
    batch - int. id партии денег полученной дропом.
    """
    amount_received = drop_client_act.received(batch=batch)
    scenario = np.random.choice(["atm-only", "transfer+atm", "transfer", "transfer_chunks"])
    for i in range(n):
        if i == 0:
            first = True
        else:
            first = False
        
        if scenario == "atm-only":
            atm_withdrawal()
            
        elif scenario == "atm+transfer":
            atm_withdrawal()

## Функция генерации активности дропа `drop_activity`

In [ ]:
def drop_activity(drop_info, drop_client_act: DropClient, lifetime):
    """
    drop_info - namedtuples из itertuples(). Информация клиента-дропа из общего датафрейма клиентов
    drop_client_act - DropClient. Генератор атрибутов дропа и его действий; генератор счетов транзакций дропа.
    lifetime - int. Сколько дней дроп будет проявлять активность, учитывая даже отклоненные транзакции
    """
    amount_received = drop_client_act.received(batch=)
    scenario = np.random.choice(["atm-only", "transfer+atm", "transfer", "transfer_chunks"])
    for i in range(n):
        if i == 0:
            first = True
        else:
            first = False
        
        if scenario == "atm-only":
            atm_withdrawal()
            
        elif scenario == "atm+transfer":
            atm_withdrawal()

In [1077]:
?DropClient.receive

Signature:
DropClient.receive(
    self,
    batch,
    low=5000,
    high=100000,
    mean=30000,
    std=20000,
    round=500,
)
Docstring:
Генерация суммы входящего перевода
--------------------------
batch - int. id текущей партии денег, которую он должен распределить (снять, перевести)
low - float. Минимальная сумма
high - float. Максимальная сумма
mean - float. Средняя сумма
std - float. Стандартное отклонение
round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце.
             При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1.
File:      c:\users\iaros\appdata\local\temp\ipykernel_7648\484414114.py
Type:      function

## Функция генерации входящих/исходящих переводов дропов

In [ ]:
def drop_transfer(type, first, configs: ConfigForTrans, fraud_amount: TransAmount, \
                 trans_part_data: FraudTransPartialData, last_txn_unix, accounts, lag_interval=None) -> dict:
    
	if type == 'inbound':
		fraud_amount.transfer(type=type)
		fraud_amount.balance += amount
		trans_part_data.original_data(online=True)
        
	elif type == 'outbound':
		TransAmount.transfer(type=type)
		TransAmount.balance -= amount
		trans_part_data.original_data(online=True)

	if first:
        timestamp_sample = configs.timestamps.sample(1)
		txn_time = timestamp_sample.timestamp.iat[0]
        txn_unix = timestamp_sample.unix_time.iat[0]
	else:
		txn_time, txn_unix = derive_from_last_time(last_txn_unix=last_txn_unix, lag_interval=lag_interval)

    account = accounts.sample(1).iat[0]

	build_trans()


In [820]:
?get_time_for_trans

Signature:
get_time_for_trans(
    trans_df,
    is_fraud,
    time_weights,
    timestamps,
    timestamps_1st_month,
    round_clock,
    online=None,
    rule=None,
    geo_distance=None,
    lag=None,
)
Docstring:
trans_df - датафрейм с транзакциями текущего клиента. Откуда брать информацию по предыдущим транзакциям клиента
is_fraud - boolean. Фрод или не фрод
time_weights - датафрейм с весами часов в периоде времени
timestamps - датафрейм с timestamps
timestamps_1st_month - сабсет timestamps отфильтрованный по первому месяцу и, если применимо, году. Чтобы генерировать первые транзакции
round_clock - boolean. Круглосуточная или дневная категория.
online - boolean. Онлайн или оффлайн покупка. True or False
rule - str. Название антифрод правила
geo_distance - int. Дистанция между локацией последней и текущей транзакции если фрод со сменой геолокации - в километрах
lag - boolean. Задержка по времени от предыдущей транзакции. Нужна для моделирования увеличения частоты транзакций.
     

# Урезание количества клиентов - `отменено`
**Думаю просто можно равновероятно семплировать n клиентов и все, без ухищрений как внизу**  
Т.к. на ~5400 клиентов нужно очень много транзакций, чтобы у каждого их было хотя бы 50.  
Условно 270000 транзакций по 50 на 5400 клиентов. Скорее всего генерироваться будут больше часа

In [436]:
# на данный момент примерно до 800.

# clients_reduced_num = pd.DataFrame(districts_ru.clients.mul(0.15).round().astype("int"))
# clients_reduced_num["city"] = districts_ru["area"].copy()
# clients_reduced_num

# clients_reduced_df = gpd.GeoDataFrame(columns=clients_with_geo.columns)
# # clients_reduced_df.columns = 
# for col, data_type in clients_with_geo.dtypes.items():
#     # if data_type != "geometry" print(data_type)
#     clients_reduced_df[col] = clients_reduced_df[col].astype(f"{data_type}")
# # list(zip(clients_with_geo.dtypes.items(), clients_reduced_df.columns))
# clients_reduced_df.dtypes

# for row in clients_reduced_num.itertuples():
#     clients_with_geo_part = clients_with_geo.loc[clients_with_geo["area"] == row.city].iloc[0:row.clients]
#     clients_reduced_df = pd.concat([clients_reduced_df,clients_with_geo_part])

# clients_reduced_df

# Старый код, функции и т.д.

## Класс `DropClient`
- метод get_account(self): Мб добавить вариант to_drop: bool. Перевод другому имеющемуся дропу. при этом. 
    - записываем счет дропа в свой счет. Атрибут account
    - записываем  счет дропа в атрибут drop_accounts
    - если надо перевести другому дропу, то фильтруем drop_accounts исключая свой
    - надо тогда клиентам приписать шестизначные счета. просто по порядку.
    - счета куда отправляют дропы это номера начинающиеся со счета последнего клиента + 1

In [ ]:
# class DropClient: 
#     """
#     Генератор сумм входящих/исходящих транзакций, сумм снятий, номеров счетов.
#     Управление балансом текущего дропа.
#     """
#     def __init__(self, accounts, account, outer_accounts, balance=0, batch_txns=0, chunk_size=0, used_accounts=pd.Series(name="account_id")):
#         """
#         accounts - pd.DataFrame. Счета клиентов банка.
#         account - int. Номер счета текущего дропа.
#         outer_accounts - pd.Series. Номера счетов для входящих и исходящих переводов в/из других банков.
#         balance - float. Текущий баланс дропа
#         batch_txns - int. Счетчик транзакций сделанных в рамках распределения полученной партии денег
#         chunk_size - int, float. Последний созданный размер части баланса для перевода по частям.
#         used_accounts - pd.Series. Счета на которые дропы уже отправляли деньги.
#         """
#         self.accounts = accounts
#         self.account = account
#         self.outer_accounts = outer_accounts
#         self.balance = balance
#         self.batch_txns = batch_txns
#         self.chunk_size = chunk_size
#         self.used_accounts = used_accounts

#     def update_balance(self, amount, add=False, declined=False):
#         """
#         Увеличить/уменьшить баланс на указанную сумму
#         -------------------
#         amount - float, int.
#         add - bool. Прибавлять сумму или отнимать.
#         declined - bool. Отклонена ли транзакция или одобрена.
#         """
#         # Не обновлять баланс если транзакция отклонена.
#         if declined:
#             return
            
#         # Увеличить баланс   
#         if add:
#             self.balance += amount
#             return
            
#         # Уменьшить баланс    
#         self.balance -= amount

#     def receive(self, declined, low=5000, high=100000, mean=30000, std=20000, round=500):
#         """
#         Генерация суммы входящего перевода
#         --------------------------
#         declined - bool. Отклонена ли транзакция или одобрена
#         low - float. Минимальная сумма
#         high - float. Максимальная сумма
#         mean - float. Средняя сумма
#         std - float. Стандартное отклонение
#         round - int. Округление целой части. По умолчанию 500. Значит числа будут либо с 500 либо с 000 на конце
#                      При условии что round не больше low и high. Чтобы отменить округление, нужно выставить 1
#         """

#         # Генерация суммы. Округление целой части при необходимости
#         amount = get_values_from_truncnorm(low_bound=low, high_bound=high, mean=mean, std=std)[0] // round * round
        
#         # Обновляем баланс если транзакция не отклонена
#         self.update_balance(amount=amount, add=True, declined=declined)
        
#         return amount

#     def get_chunk_size(self, online=False, atm_min=10000, atm_share=0.5, round=500, rand_rate=0.9, start=0, stop=0, step=0):
#         """
#         Вернуть случайный размер суммы перевода для перевода по частям
#         либо вернуть долю от баланса для снятия/перевода по частям.
#         -------------------------------
#         online - bool. Онлайн или оффлайн. Перевод или банкомат. Если банкомат, то снимается доля atm_share от баланса, но не меньше atm_min
#         atm_min - int, float. Минимальная сумма снятия дропом в банкомате.
#         atm_share - float. Доля от баланса если снятие через банкомат.
#         round - int. Округление целой части. По умолчанию 500. 
#                      Значит суммы будут округлены до тысяч или пяти сотен
#         rand_rate - float. От 0 до 1. Процент случаев, когда каждый НЕ первый чанк будет случайным и не зависеть от предыдущего.
#                            Но возможны случайные совпадаения с предыдущим размером чанка.
#                            Доля случайных размеров подряд будет:
#                            p(c) - вероятность взять определенный размер (зависит от размера выборки чанков)
#                            p(rr) - rand_rate
#                            p(rr) - (p(rr) * p(c)). Например p(rr) = 0.9, 5 вариантов размеров чанка - p(c) = 0.20
#                            0.9 - (0.9 * 0.2) = 0.72
#                            В около 72% случаев размеры чанков не будут подряд одинаковыми 
#         start - int. Минимальный размер. Прописываем если генерация не через share.
#                      То же самое для stop и step
#         stop - int. Максимальный размер - не входит в возможный выбор.
#                     Максимальное генерируемое значение равно stop - step
#         step - int. Шаг размеров.
#         --------------------
#         Возвращает np.int64
#         Результат кэшируется в self.chunk_size
#         """
#         # Если это не первая транзакция в серии транзакции для одной полученной дропом суммы
#         # И случайное число больше rand_rate, то просто возвращаем ранее созданный размер чанка
#         if self.batch_txns != 0 and np.random.uniform(0,1) > rand_rate:
#             return self.chunk_size

#         # Если перевод
#         if online:
#             sampling_array = np.arange(start, stop, step)
#             self.chunk_size = np.random.choice(sampling_array)
#             return self.chunk_size
            
#         # Если снятие    
#         self.chunk_size = max(atm_min, self.balance * atm_share // round * round)
#         return self.chunk_size
            
        
#     def one_operation(self, amount=0, declined=False, in_chunks=False):
#         """
#         Генерация суммы операции дропа.
#         ---------
#         amount - float, int. Сумма перевода если перевод по частям - in_chunks == True
#         declined - bool. Отклонена ли транзакция или одобрена
#         in_chunks - bool. Перевод по частям или целиком. Если False, то просто пробуем перевести все с баланса
#                           При True нужно указать amount.
#         """
#         if in_chunks and amount <= 0:
#             raise ValueError(f"""If in_chunks is True, then amount must be greater than 0.
# Passed amount: {amount}""")

#         # Если перевод не по частям. Пробуем перевестит все с баланса. 
#         if not in_chunks:
#             amount = self.balance
#             self.update_balance(amount=self.balance, add=False, declined=declined)
#             # Прибавляем счетчик транзакции для текущей партии денег
#             self.batch_txns += 1
#             return amount

#         # Иначе считаем сколько частей исходя из размера одной части
#         chunks = self.balance // amount

#         # Если целое число частей больше 0. Пробуем перевести одну часть
#         if chunks > 0:
#             self.update_balance(amount=amount, add=False, declined=declined)
#             self.batch_txns += 1
#             return amount

#         # Если баланс меньше одной части. Пробуем перевести то что осталось
#         rest = self.balance
#         self.update_balance(amount=rest, add=False, declined=declined)
#         self.batch_txns += 1
#         return rest

#     def get_account(self, to_drop):
#         """
#         Номер счета входящего/исходящего перевода
#         to_drop - bool. Перевод другому дропу в нашем банке или нет.
#         """
#         # Фильтруем accounts исключая свой счет и выбирая дропов. Для случая если to_drop
#         drop_accounts = self.accounts.loc[(self.accounts.account_id != self.account) & (self.accounts.is_drop == True)]

#         # Если надо отправить другому дропу в нашем банке. При условии что есть другие дропы на текущий момент
#         if to_drop and not drop_accounts.empty: 
#             account = drop_accounts.account_id.sample(1).iat[0]
#             # Добавляем этот счет в использованные как последнюю запись в серии
#             self.used_accounts.loc[self.used_accounts.shape[0]] = account
#             return account

#         # Если отправляем/получаем из другого банка.  
#         # Семплируем номер внешнего счета который еще не использовался
#         account = self.outer_accounts.loc[~(self.outer_accounts.isin(self.used_accounts))].sample(1).iat[0]
#         # Добавляем этот счет в использованные как последнюю запись в серии
#         self.used_accounts.loc[self.used_accounts.shape[0]] = account
        
#         return account

#     def reset_cache(self, balance=True, used_accounts=True, chunk_size=True, batch_txns=True):
#         """
#         Сброс кэшированных значений
#         По умолчанию сбрасывается всё. Если что-то надо оставить, то надо выставить False
#         для этого
#         -----------------
#         balance - bool
#         used_accounts - bool
#         chunk_size - bool
#         batch_txns - bool
#         """
#         if balance:
#             self.balance = 0
#         if used_accounts:
#             self.used_accounts = pd.Series(name="account_id")
#         if chunk_size:
#             self.chunk_size = 0
#         if batch_txns:
#             self.batch_txns = 0